In [ ]:
!pip install -I xpander-sdk openai python-dotenv

In [2]:
from xpander_sdk import XpanderClient
from openai import OpenAI
from dotenv import load_dotenv
import os
from IPython.display import Markdown

## Load environment variables
load_dotenv()
friendli_client = OpenAI(
    base_url="https://api.friendli.ai/dedicated",
    api_key=os.environ.get("FRIENDLI_TOKEN")
)

xpanderAPIKey = os.environ.get("XPANDER_API_KEY","")
xpanderAgentID = os.environ.get("XPANDER_AGENT_ID", "")


xpander_client = XpanderClient(api_key=xpanderAPIKey)
xpander_agent = xpander_client.agents.get(agent_id=xpanderAgentID)

tools = xpander_agent.retrieve_all_graph_tools()
print(tools)


[{'type': 'function', 'function': {'name': 'LinkedInCompanyManagementGetCompanyDetailsByUsername', 'description': "Retrieves comprehensive information about a company using its LinkedIn username. This operation provides valuable insights into company profiles, including size, industry, and location. If the username isn't provided, run GetCompanyInfoByDomain if you have the company's domain name. Use this to gather detailed company information for business intelligence, partnership opportunities, or competitive analysis.", 'parameters': {'type': 'object', 'properties': {'bodyParams': {'type': 'object', 'properties': {}, 'required': []}, 'queryParams': {'type': 'object', 'properties': {'username': {'type': 'string', 'description': "Company's LinkedIn username (e.g., 'google')."}}, 'required': []}, 'pathParams': {'type': 'object', 'properties': {}, 'required': []}}, 'required': ['bodyParams', 'queryParams', 'pathParams']}}}, {'type': 'function', 'function': {'name': 'PerplexityChatComplet

In [3]:
# verify the friendli client is working
completion = friendli_client.chat.completions.create(
    model="meta-llama-3.1-8b-instruct",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello!"}
    ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content='Hello. What can I help you with today? Do you have any questions, need information on a particular topic, or is there something else I can assist you with?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


In [6]:
memory = [{"role": "system", "content": "You are a helpful assistant. You are running in While True Loop, the response to your tools are available, once you have completed the user request add ##FINAL ANSWER## in your reply without further function calling"},
          {"role": "user", "content": "Send hi to C08628AHP9C channel"}]
print(memory[-1])
number_of_calls = 1
while True:
    tools = xpander_agent.retrieve_all_graph_tools()
    llm_response = friendli_client.chat.completions.create(
        model="meta-llama-3.1-70b-instruct",
        messages=memory,
        tools=tools
        )
    memory.append({"role": "assistant", "content": f'Step number: {number_of_calls}'})
    model_response = llm_response.choices[0].message
    print(model_response)
    memory.append({"role": "assistant", "content": model_response.content})
    if model_response.tool_calls:
        print("model decided to invoke tools")
        tools_to_run = XpanderClient.extract_tool_calls(llm_response=llm_response.model_dump())
        for tool in tools_to_run: 
            print(tool.name)
        tool_responses = xpander_agent.run_tools(tool_calls=tools_to_run)
        for tool_response in tool_responses:
            print("tool response for", tool_response.function_name, tool_response.status_code)
            memory.append({"role": "tool", "content": tool_response.result, "tool_call_id": tool_response.tool_call_id})
    if (model_response.content):
        if "##FINAL ANSWER##" in model_response.content:
            break
    number_of_calls += 1
print(model_response.content)   

{'role': 'user', 'content': 'Send hi to C08628AHP9C channel'}
ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_WcYhGOR0FfuSpHRnTfBBbHGO', function=Function(arguments='{"bodyParams": {"channel": "C08628AHP9C", "text": "hi", "unfurl_links": false, "unfurl_media": false, "username": "hi-bot"}, "queryParams": {}, "pathParams": {}}', name='SlackMessageSendingSendMessageToChannel'), type='function')])
model decided to invoke tools
SlackMessageSendingSendMessageToChannel
tool response for SlackMessageSendingSendMessageToChannel 200
ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_ensF5YRlPmjnnEKPjld7nx9r', function=Function(arguments='{"bodyParams": {"channel": "C08628AHP9C", "text": "hi", "username": "My Slack Bot"}, "queryParams": {}, "pathParams": {}}', name='SlackMessageSendingSendMessage